#### ========================================
### NOTEBOOK 2 : PRÉTRAITEMENT DU TEXTE
### Projet Classification de CV - LiveCareer
### ========================================

### Prétraitement du Texte des CV
 
##### **Ce notebook utilise la classe TextCleaner de src/preprocessing/text_cleaner.py**

In [ ]:
### 1. Imports et Configuration

In [ ]:

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import sys
from pathlib import Path

# Ajouter le dossier src au path Python
sys.path.append(str(Path.cwd().parent))

# Importer notre classe TextCleaner depuis src/
from src.preprocessing.text_cleaner import TextCleaner

import warnings
warnings.filterwarnings('ignore')

print("✅ Imports réussis!")
print(f"   - TextCleaner importé depuis: src/preprocessing/text_cleaner.py")

# %% [markdown]
# ## 2. Chargement des Données Brutes

# %%
# Charger le dataset original
df = pd.read_csv('../data/raw/resume_dataset.csv')

print(f"📊 Dataset chargé:")
print(f"   - Nombre de CV: {len(df)}")
print(f"   - Colonnes: {df.columns.tolist()}")

# Identifier les colonnes
text_col = 'Resume'  # Ajustez selon votre dataset
category_col = 'Category'

# Afficher un exemple
print("\n📄 Exemple de CV brut (500 premiers caractères):")
print("="*80)
print(df[text_col].iloc[0][:500] + "...")

# %% [markdown]
# ## 3. Initialisation du TextCleaner

# %%
# Créer une instance du TextCleaner
# (La classe est définie dans src/preprocessing/text_cleaner.py)
cleaner = TextCleaner(
    lowercase=True,
    remove_urls=True,
    remove_emails=True,
    remove_phone_numbers=True,
    remove_numbers=False,  # Garder les années d'expérience
    remove_punctuation=True,
    remove_stopwords=True,
    lemmatize=True,
    stem=False
)

print("✅ TextCleaner initialisé avec les paramètres:")
print(f"   - Stopwords: {len(cleaner.stop_words)} mots")

# %% [markdown]
# ## 4. Test sur un Exemple

# %%
# Prendre un CV exemple
sample_cv = df[text_col].iloc[0]

print("📝 AVANT NETTOYAGE:")
print("="*80)
print(sample_cv[:500] + "...")

# Nettoyer avec notre classe
cleaned_sample = cleaner.clean_text(sample_cv)

print("\n✨ APRÈS NETTOYAGE:")
print("="*80)
print(cleaned_sample[:500] + "...")

print(f"\n📊 STATISTIQUES:")
print(f"   Longueur originale: {len(sample_cv)} caractères")
print(f"   Longueur nettoyée:  {len(cleaned_sample)} caractères")
print(f"   Réduction: {((len(sample_cv) - len(cleaned_sample)) / len(sample_cv)) * 100:.1f}%")

# %% [markdown]
# ## 5. Nettoyage de Tout le Dataset

# %%
# Utiliser la méthode clean_dataframe de notre classe
df = cleaner.clean_dataframe(df, text_column=text_col, output_column='cleaned_text')

# Afficher les premières lignes
print("\n👀 Aperçu du DataFrame nettoyé:")
df[[category_col, 'cleaned_text']].head()

# %% [markdown]
# ## 6. Analyse du Nettoyage

# %%
# Calculer les longueurs avant/après
df['length_before'] = df[text_col].astype(str).apply(len)
df['length_after'] = df['cleaned_text'].apply(len)
df['reduction_percent'] = ((df['length_before'] - df['length_after']) / df['length_before']) * 100

print("\n📊 STATISTIQUES DE NETTOYAGE:")
print("="*60)
print(df[['length_before', 'length_after', 'reduction_percent']].describe())

# %% [markdown]
# ## 7. Visualisation de l'Impact du Nettoyage

# %%
# Créer des visualisations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Distribution longueurs avant
axes[0, 0].hist(df['length_before'], bins=50, color='lightcoral', 
               edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Longueur (caractères)', fontweight='bold')
axes[0, 0].set_ylabel('Fréquence', fontweight='bold')
axes[0, 0].set_title('Distribution AVANT Nettoyage', fontsize=14, fontweight='bold')
axes[0, 0].axvline(df['length_before'].mean(), color='red', linestyle='--', 
                   label=f'Moyenne: {df["length_before"].mean():.0f}')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Distribution longueurs après
axes[0, 1].hist(df['length_after'], bins=50, color='lightgreen', 
               edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Longueur (caractères)', fontweight='bold')
axes[0, 1].set_ylabel('Fréquence', fontweight='bold')
axes[0, 1].set_title('Distribution APRÈS Nettoyage', fontsize=14, fontweight='bold')
axes[0, 1].axvline(df['length_after'].mean(), color='green', linestyle='--',
                   label=f'Moyenne: {df["length_after"].mean():.0f}')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 3. Réduction en pourcentage
axes[1, 0].hist(df['reduction_percent'], bins=50, color='skyblue',
               edgecolor='black', alpha=0.7)
axes[1, 0].set_xlabel('Réduction (%)', fontweight='bold')
axes[1, 0].set_ylabel('Fréquence', fontweight='bold')
axes[1, 0].set_title('Taux de Réduction', fontsize=14, fontweight='bold')
axes[1, 0].axvline(df['reduction_percent'].mean(), color='blue', linestyle='--',
                   label=f'Moyenne: {df["reduction_percent"].mean():.1f}%')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 4. Scatter avant vs après
sample_df = df.sample(min(1000, len(df)))
axes[1, 1].scatter(sample_df['length_before'], sample_df['length_after'],
                  alpha=0.5, c='purple')
axes[1, 1].set_xlabel('Longueur AVANT', fontweight='bold')
axes[1, 1].set_ylabel('Longueur APRÈS', fontweight='bold')
axes[1, 1].set_title('Corrélation Avant/Après', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

# Ligne de référence
max_val = max(df['length_before'].max(), df['length_after'].max())
axes[1, 1].plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='Pas de réduction')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('../outputs/plots/cleaning_impact.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique sauvegardé dans outputs/plots/cleaning_impact.png")

# %% [markdown]
# ## 8. Analyse des Mots les Plus Fréquents

# %%
# Utiliser la méthode get_word_frequency de notre classe
word_freq = cleaner.get_word_frequency(df['cleaned_text'], top_n=30)

print("\n🔤 TOP 30 MOTS APRÈS NETTOYAGE:")
print("="*60)
for i, (word, count) in enumerate(word_freq.items(), 1):
    print(f"{i:2d}. {word:20s} : {count:7d}")

# %% [markdown]
# ## 9. Comparaison Avant/Après sur un Exemple

# %%
# Utiliser la méthode visualize_cleaning_effect de notre classe
print("\n" + "="*80)
print("📊 EXEMPLE DE NETTOYAGE DÉTAILLÉ")
print("="*80)

sample_idx = 5
original = df[text_col].iloc[sample_idx]
cleaned = df['cleaned_text'].iloc[sample_idx]

cleaner.visualize_cleaning_effect(original, cleaned)

# %% [markdown]
# ## 10. Sauvegarde des Données Nettoyées

# %%
# Sauvegarder le DataFrame nettoyé
output_path = '../data/processed/resume_cleaned.csv'
df.to_csv(output_path, index=False)

print(f"💾 Données nettoyées sauvegardées dans: {output_path}")
print(f"   Colonnes sauvegardées: {df.columns.tolist()}")
print(f"   Nombre de lignes: {len(df)}")

# Version compacte (seulement les colonnes essentielles)
df_compact = df[[category_col, 'cleaned_text']].copy()
df_compact.to_csv('../data/processed/resume_cleaned_compact.csv', index=False)

print(f"\n💾 Version compacte sauvegardée dans: resume_cleaned_compact.csv")

# %% [markdown]
# ## 11. Résumé

# %%
print("\n" + "="*80)
print("📊 RÉSUMÉ DU PRÉTRAITEMENT")
print("="*80)

print(f"""
✅ DONNÉES TRAITÉES:
   - CV nettoyés: {len(df)}
   - Réduction moyenne: {df['reduction_percent'].mean():.1f}%
   - Longueur moyenne avant: {df['length_before'].mean():.0f} caractères
   - Longueur moyenne après: {df['length_after'].mean():.0f} caractères

📁 FICHIERS CRÉÉS:
   - data/processed/resume_cleaned.csv (complet)
   - data/processed/resume_cleaned_compact.csv (compact)
   - outputs/plots/cleaning_impact.png

🔧 CLASSE UTILISÉE:
   - src/preprocessing/text_cleaner.py → TextCleaner

🎯 PROCHAINES ÉTAPES:
   1. Feature Extraction (TF-IDF) → Notebook 03
   2. Utilise: src/preprocessing/feature_extractor.py
   
✅ Passez au Notebook 03_feature_extraction.ipynb
""")

# %% [markdown]
# ---
# ## ✅ FIN DU PRÉTRAITEMENT
# 
# **Points clés:**
# - ✅ Utilisé TextCleaner depuis src/preprocessing/text_cleaner.py
# - ✅ Pas de duplication de code
# - ✅ Fichier resume_cleaned.csv créé
# - ✅ Prêt pour l'extraction de features